In [1]:
import re
import pandas as pd

#language detector
from langdetect import detect
from langdetect import detect_langs

In [2]:
df = pd.read_csv("../crawling/merged_csv.csv")

In [3]:
print(df.shape)
df.head()

(11101, 5)


,Unnamed: 0,Unnamed: 0.1,category,header,content
0,0,0,politics,Mary Trump slams Susan Sarandon as a 'moron' f...,Mary Trump criticized longtime activist and ac...
1,1,1,politics,Moment two Ukrainian fighter jets bomb Russian...,New footage shows the moment that Ukrainian A...
2,2,2,politics,Mark Esper on what's at stake if Donald Trump ...,Former Secretary of Defense Mark Esper was con...
3,3,3,politics,Hunter Biden enlists Hollywood mega-lawyer for...,Hunter Biden has garnered quiet support and fi...
4,4,4,politics,President Trump's response to Mark Esper's 60 ...,When asked for comment on this week's intervie...


In [4]:
df.drop(columns="Unnamed: 0", axis=1, inplace=True)
df.drop(columns="Unnamed: 0.1", axis=1, inplace=True)

In [5]:
df

,category,header,content
0,politics,Mary Trump slams Susan Sarandon as a 'moron' f...,Mary Trump criticized longtime activist and ac...
1,politics,Moment two Ukrainian fighter jets bomb Russian...,New footage shows the moment that Ukrainian A...
2,politics,Mark Esper on what's at stake if Donald Trump ...,Former Secretary of Defense Mark Esper was con...
3,politics,Hunter Biden enlists Hollywood mega-lawyer for...,Hunter Biden has garnered quiet support and fi...
4,politics,President Trump's response to Mark Esper's 60 ...,When asked for comment on this week's intervie...
...,...,...,...
11096,food,Classic Tater Tot Casserole with Green Beans,This is one of my favorite childhood recipe's....
11097,food,Stews of South India,Know about different stews present in South In...
11098,food,Biscuits from Self Rising Flour,Biscuits are a learning experience. Best buy e...
11099,food,Streusel Topping for Banana Bread for Example,Banana bread topping is added just before you ...


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11101 entries, 0 to 11100
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   category  11101 non-null  object
 1   header    11101 non-null  object
 2   content   9499 non-null   object
dtypes: object(3)
memory usage: 260.3+ KB


In [7]:
df[df['content'].isna()]

,category,header,content
211,technology,Technology to Make Your Life Easier,NaN
214,technology,Steps to Automating Your Production Plant,NaN
240,technology,How to Cancel a Subscription on iPhone,NaN
243,technology,What Tools Are Needed for 5G mmWave Testing?,NaN
246,technology,TRXmining : The Next Decentralized Finance Hub,NaN
...,...,...,...
11052,food,Kung Pao Chicken Introduction,NaN
11053,food,One of the Most Popular Snack and Dessert Tofu...,NaN
11054,food,Traditional Chinese Snack -- Stinky Tofu,NaN
11093,food,No Flour Chocolate Cake,NaN


In [8]:
df.dropna(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9499 entries, 0 to 11100
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   category  9499 non-null   object
 1   header    9499 non-null   object
 2   content   9499 non-null   object
dtypes: object(3)
memory usage: 296.8+ KB


In [9]:
df[df.duplicated() == True]

,category,header,content
753,technology,Motivational Sayings Great Ones,Why examine motivational sayings? For motivati...
754,technology,5 Reasons Why the Fire TV Stick 4k Is the Top ...,The Fire TV Stick 4K has quickly become the to...
755,technology,The Anker PowerConf H700 AI-Powered Wireless H...,A headset that mitigates noise and lets you ta...
756,technology,Four Gadgets that will Enhance Your Cycling Ex...,"Friends, through this article of today, we are..."
757,technology,Headphones On and World Off With Happy Plugs A...,The Happy Plugs Air 1 ANC True Wireless Earbud...
...,...,...,...
11088,food,"When Life Gives You Lemons, Reap Their Health ...",From preventing heart disease and cancer to im...
11089,food,Poi Besar (Malabar spinach mustard curry / Bas...,"The end of spring and beginning of summer, the..."
11090,food,Matar Paneer - Indian Cuisine,Matar Paneer - In Hindi Matar means Green peas...
11091,food,What Is Structured Water and How to Prepare it...,"Learn what structured water is, how to make it..."


In [10]:
df.drop_duplicates(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6710 entries, 0 to 11100
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   category  6710 non-null   object
 1   header    6710 non-null   object
 2   content   6710 non-null   object
dtypes: object(3)
memory usage: 209.7+ KB


In [11]:
#drop < 5 words
df = df[df["content"].str.split().str.len().gt(5)]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6595 entries, 0 to 11100
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   category  6595 non-null   object
 1   header    6595 non-null   object
 2   content   6595 non-null   object
dtypes: object(3)
memory usage: 206.1+ KB


In [12]:
# Remove numbers and \n \r - 
def remove_numbers_and_backslashes(column_name):
    for i, row in df.iterrows():    
        row[f"{column_name}"] = re.sub(r'\d','', row[f"{column_name}"])
        row[f"{column_name}"] = row[f"{column_name}"].replace("\n"," ")
        row[f"{column_name}"] = row[f"{column_name}"].replace("\r"," ")
        row[f"{column_name}"] = row[f"{column_name}"].replace("-"," ")


remove_numbers_and_backslashes("header")
remove_numbers_and_backslashes("content")

,category,header,content
0,politics,Mary Trump slams Susan Sarandon as a 'moron' f...,Mary Trump criticized longtime activist and ac...
1,politics,Moment two Ukrainian fighter jets bomb Russian...,New footage shows the moment that Ukrainian A...
2,politics,Mark Esper on what's at stake if Donald Trump ...,Former Secretary of Defense Mark Esper was con...
3,politics,Hunter Biden enlists Hollywood mega lawyer for...,Hunter Biden has garnered quiet support and fi...
4,politics,President Trump's response to Mark Esper's Mi...,When asked for comment on this week's intervie...


In [14]:
df.shape

(6595, 3)

In [15]:
for i, row in df.iterrows():
    try:
        # if en in detect_langs continue
        content_language = detect(row["content"])
        if (row["category"] != "food") and (content_language != "en"):
            print("--------------------------------------")
            print(f"!!! Detect content as {content_language} language.\nDrop row form df:\n{row}")
            print(detect_langs(row["content"]))
            print("--------------------------------------")
            print()
            df.drop(i, inplace=True)
    
    except Exception as error:
        print(f"Drop row form df:\n {row}")
        df.drop(i, inplace=True)    

-------------------
!!! Detect content as no language.
Drop row form df:
category                                           technology
header                               Top  Best Eyelid Tape ()
content     best eyelid tape for aging eyes best eyelid ta...
Name: 299, dtype: object
[no:0.5609351257616053, en:0.1533512705800433, tr:0.14285682986994255, af:0.14285602477450782]
!!!!!!!!!!!!!!!!!!!!



/Users/sergeyjuchenko/opt/anaconda3/lib/python3.9/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


-------------------
!!! Detect content as no language.
Drop row form df:
category                                   technology
header      Emails   Deleting from Gmail   One Method
content            I have to start deleting emails...
Name: 376, dtype: object
[no:0.5714258747320873, en:0.428570331474819]
!!!!!!!!!!!!!!!!!!!!

-------------------
!!! Detect content as pt language.
Drop row form df:
category                                              science
header                                           Quem sou eu?
content     Além da aspirante a poeta que já publicou dive...
Name: 1322, dtype: object
[pt:0.9999947313697295]
!!!!!!!!!!!!!!!!!!!!

-------------------
!!! Detect content as hi language.
Drop row form df:
category                                              science
header      Tissue In Hindi Science Class th Chapter  | ऊत...
content     अगर आप  वी विज्ञान (th Science) के छात्र है तो...
Name: 1373, dtype: object
[hi:0.9999956604846301]
!!!!!!!!!!!!!!!!!!!!

----------

-------------------
!!! Detect content as ko language.
Drop row form df:
category                                              science
header                                             我們比黑猩猩還聰明﹖
content     Ben Ambridge《我們比黑猩猩還聰明﹖》，周沛郁譯，台北﹕天下文化， 書的內容其實不...
Name: 3778, dtype: object
[zh-tw:0.8571416134072406, ko:0.14285711991592917]
!!!!!!!!!!!!!!!!!!!!

-------------------
!!! Detect content as de language.
Drop row form df:
category                                          politics
header         Businesses Worried About Possible Lockdown.
content     is Wishy Washing Listening Or Even Being Fair?
Name: 4087, dtype: object
[de:0.857139319930041, en:0.14286022192745623]
!!!!!!!!!!!!!!!!!!!!

-------------------
!!! Detect content as ja language.
Drop row form df:
category                                           technology
header           IOSTSWAPファーム、エントロバース・エコシステムに初のクロスチェーンDeFiが登場
content     免責事項：IOSTSwapファームはサードパーティの開発者によって開発されました。ここで提供...
Name: 5087, dtype: object
[en:0.571428

In [16]:
print(df.shape)

(6554, 3)


In [19]:
pd.set_option('display.max_rows', None)
df


,category,header,content
0,politics,Mary Trump slams Susan Sarandon as a 'moron' f...,Mary Trump criticized longtime activist and ac...
1,politics,Moment two Ukrainian fighter jets bomb Russian...,New footage shows the moment that Ukrainian A...
2,politics,Mark Esper on what's at stake if Donald Trump ...,Former Secretary of Defense Mark Esper was con...
3,politics,Hunter Biden enlists Hollywood mega lawyer for...,Hunter Biden has garnered quiet support and fi...
4,politics,President Trump's response to Mark Esper's Mi...,When asked for comment on this week's intervie...
5,politics,National gun surrender launched,A new national gun surrender will allow people...
6,politics,"Transcript: House Speaker Nancy Pelosi on ""Fac...",The following is a transcript of an interview ...
7,politics,Mississippi Gov.: ‘Empathetic’ towards 'ladies...,"In an interview with Meet the Press, Gov. Tate..."
8,politics,Manchin says he'd pass parts of Biden's agenda...,WASHINGTON — Sen. Joe Manchin says he’s still ...
9,politics,Russia's space agency chief claimed his nation...,"Dmitry Rogozin, who leads Roscomos, has previo..."
